# Lab: Function Calling and Structured Outputs

## Learning Objectives

- Implement function calling with JSON Schema validation
- Create structured output generators with proper constraints
- Build cross-platform solutions using HuggingFace and local endpoints
- Evaluate and compare different AI providers
- Handle errors and edge cases gracefully

## Prerequisites

Install required packages:

In [ ]:
# Install required packages
!pip install openai huggingface_hub transformers jsonschema requests python-dotenv

## Setup and Configuration

Let's start by setting up our environment and API credentials:

In [ ]:
import os
import json
import time
import random
import requests
from typing import Dict, Any, List, Optional
from jsonschema import validate, ValidationError
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# API Keys (use environment variables in production)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "your-openai-key-here")
HF_TOKEN = os.getenv("HF_TOKEN", "your-huggingface-token-here")

print("Environment setup complete!")
print(f"OpenAI API Key configured: {'Yes' if OPENAI_API_KEY != 'your-openai-key-here' else 'No'}")
print(f"HuggingFace Token configured: {'Yes' if HF_TOKEN != 'your-huggingface-token-here' else 'No'}")

## Exercise 1: Basic Function Calling Setup

Let's start with a simple function calling example using a coffee recipe generator:

In [ ]:
# Define our function implementations
def make_coffee(coffee_type: str) -> Dict[str, Any]:
    """Generate a coffee recipe based on type."""
    recipes = {
        "espresso": {
            "coffee_grams": 18,
            "water_ml": 36,
            "brew_time_seconds": 25,
            "temperature_celsius": 93,
            "pressure_bar": 9
        },
        "cappuccino": {
            "coffee_grams": 18,
            "water_ml": 36,
            "milk_ml": 120,
            "milk_foam": "thick",
            "brew_time_seconds": 25
        },
        "latte": {
            "coffee_grams": 18,
            "water_ml": 36,
            "milk_ml": 240,
            "milk_foam": "thin",
            "brew_time_seconds": 25
        },
        "americano": {
            "coffee_grams": 18,
            "water_ml": 36,
            "additional_water_ml": 120,
            "brew_time_seconds": 25
        }
    }
    return recipes.get(coffee_type, {"error": "Recipe not found"})

def random_coffee_fact() -> Dict[str, Any]:
    """Return a random coffee fact."""
    facts = [
        {"fact": "Coffee was first discovered in Ethiopia around 850 AD", "source": "Historical records"},
        {"fact": "Espresso means 'pressed out' in Italian", "source": "Italian etymology"},
        {"fact": "Coffee is the world's second-most traded commodity after oil", "source": "Commodity markets"},
        {"fact": "The average American consumes 3.1 cups of coffee per day", "source": "National Coffee Association"},
        {"fact": "Coffee beans are actually seeds from coffee cherries", "source": "Botanical classification"}
    ]
    return random.choice(facts)

# Define tool schemas for the AI model
coffee_tools = [
    {
        "type": "function",
        "function": {
            "name": "make_coffee",
            "description": "Generate a coffee recipe for the specified type",
            "parameters": {
                "type": "object",
                "properties": {
                    "coffee_type": {
                        "type": "string",
                        "enum": ["espresso", "cappuccino", "latte", "americano"],
                        "description": "Type of coffee to make"
                    }
                },
                "required": ["coffee_type"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "random_coffee_fact",
            "description": "Get a random interesting fact about coffee",
            "parameters": {
                "type": "object",
                "properties": {},
                "additionalProperties": False
            }
        }
    }
]

print("Function definitions created!")
print(f"Available tools: {[tool['function']['name'] for tool in coffee_tools]}")

## Exercise 2: OpenAI Function Calling Implementation

Now let's implement function calling with OpenAI's API:

In [ ]:
try:
    from openai import OpenAI
    
    # Initialize OpenAI client
    openai_client = OpenAI(api_key=OPENAI_API_KEY)
    
    def handle_openai_function_call(messages, tools):
        """Handle function calling with OpenAI API."""
        response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )
        
        response_message = response.choices[0].message
        
        # Check if the model wants to call a function
        if response_message.tool_calls:
            messages.append(response_message)
            
            # Handle each function call
            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                print(f"Calling function: {function_name}")
                print(f"Arguments: {function_args}")
                
                # Call the actual function
                if function_name == "make_coffee":
                    function_response = make_coffee(**function_args)
                elif function_name == "random_coffee_fact":
                    function_response = random_coffee_fact()
                else:
                    function_response = {"error": f"Unknown function: {function_name}"}
                
                # Add function response to messages
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": json.dumps(function_response)
                })
            
            # Get final response from model
            final_response = openai_client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages
            )
            
            return final_response.choices[0].message.content
        
        return response_message.content
    
    # Test the function calling
    messages = [
        {"role": "system", "content": "You are a helpful coffee assistant. Use the available tools to help users with coffee-related questions."},
        {"role": "user", "content": "I'd like to make a cappuccino. Can you give me the recipe?"}
    ]
    
    result = handle_openai_function_call(messages, coffee_tools)
    print("\nFinal response:")
    print(result)
    
except ImportError:
    print("OpenAI library not installed. Skipping this exercise.")
except Exception as e:
    print(f"OpenAI API error: {e}")
    print("Skipping OpenAI exercise. Proceeding with HuggingFace alternatives.")

## Exercise 3: HuggingFace Implementation

Let's implement the same functionality using HuggingFace's open-source tools:

In [ ]:
try:
    from huggingface_hub import InferenceClient
    
    # Initialize HuggingFace client
    hf_client = InferenceClient(token=HF_TOKEN)
    
    def create_structured_prompt(user_query: str, tools: List[Dict]) -> str:
        """Create a structured prompt for HuggingFace models."""
        tool_descriptions = []
        for tool in tools:
            func = tool['function']
            tool_descriptions.append(f"""
Tool: {func['name']}
Description: {func['description']}
Parameters: {json.dumps(func['parameters'], indent=2)}
            """)
        
        prompt = f"""
You are a helpful assistant with access to the following tools:

{chr(10).join(tool_descriptions)}

User Query: {user_query}

If you need to use a tool, respond with exactly this format:
TOOL_CALL: {{"name": "tool_name", "arguments": {{"param1": "value1"}}}}

If no tool is needed, respond naturally.
        """
        
        return prompt
    
    def parse_tool_call(response_text: str) -> Optional[Dict[str, Any]]:
        """Parse tool call from model response."""
        if "TOOL_CALL:" in response_text:
            try:
                # Extract JSON after TOOL_CALL:
                json_start = response_text.find("TOOL_CALL:") + len("TOOL_CALL:")
                json_str = response_text[json_start:].strip()
                return json.loads(json_str)
            except json.JSONDecodeError:
                return None
        return None
    
    def handle_hf_function_call(user_query: str, tools: List[Dict], available_functions: Dict) -> str:
        """Handle function calling with HuggingFace models."""
        prompt = create_structured_prompt(user_query, tools)
        
        # Use a conversational model
        response = hf_client.text_generation(
            prompt,
            model="microsoft/DialoGPT-medium",
            max_new_tokens=150,
            temperature=0.1
        )
        
        print(f"Model response: {response}")
        
        # Check for tool call
        tool_call = parse_tool_call(response)
        if tool_call:
            function_name = tool_call['name']
            function_args = tool_call['arguments']
            
            print(f"Calling function: {function_name}")
            print(f"Arguments: {function_args}")
            
            # Call the actual function
            if function_name in available_functions:
                function_response = available_functions[function_name](**function_args)
                
                # Create follow-up prompt with function result
                follow_up_prompt = f"""
                User Query: {user_query}
                Function Result: {json.dumps(function_response)}
                
                Provide a natural response to the user based on the function result.
                """
                
                final_response = hf_client.text_generation(
                    follow_up_prompt,
                    model="microsoft/DialoGPT-medium",
                    max_new_tokens=150,
                    temperature=0.1
                )
                
                return final_response
            else:
                return f"Error: Unknown function {function_name}"
        
        return response
    
    # Test with HuggingFace
    available_functions = {
        "make_coffee": make_coffee,
        "random_coffee_fact": random_coffee_fact
    }
    
    result = handle_hf_function_call(
        "I'd like to make a latte. Can you give me the recipe?",
        coffee_tools,
        available_functions
    )
    
    print("\nHuggingFace result:")
    print(result)
    
except ImportError:
    print("HuggingFace Hub library not installed. Skipping this exercise.")
except Exception as e:
    print(f"HuggingFace API error: {e}")

## Exercise 4: Structured Output Validation

Now let's create a robust validation system for structured outputs:

In [ ]:
def validate_json_schema(data: Dict[str, Any], schema: Dict[str, Any]) -> bool:
    """Validate data against JSON schema."""
    try:
        validate(instance=data, schema=schema)
        return True
    except ValidationError as e:
        print(f"Validation error: {e.message}")
        return False

def safe_json_parse(response_text: str) -> Optional[Dict[str, Any]]:
    """Extract and parse JSON from potentially mixed responses."""
    import re
    
    try:
        # Try direct parsing first
        return json.loads(response_text)
    except json.JSONDecodeError:
        # Extract JSON from code blocks
        json_match = re.search(r'```json\n(.*?)\n```', response_text, re.DOTALL)
        if json_match:
            return json.loads(json_match.group(1))
        # Try to find JSON-like content
        json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
        if json_match:
            return json.loads(json_match.group(0))
        return None

# Define a schema for game character generation
game_character_schema = {
    "type": "object",
    "properties": {
        "character": {
            "type": "object",
            "properties": {
                "name": {"type": "string", "minLength": 2, "maxLength": 20},
                "class": {"type": "string", "enum": ["warrior", "mage", "rogue", "cleric"]},
                "health": {"type": "integer", "minimum": 50, "maximum": 100},
                "mana": {"type": "integer", "minimum": 0, "maximum": 100},
                "strength": {"type": "integer", "minimum": 1, "maximum": 20},
                "intelligence": {"type": "integer", "minimum": 1, "maximum": 20}
            },
            "required": ["name", "class", "health"],
            "additionalProperties": False
        },
        "backstory": {
            "type": "string",
            "minLength": 50,
            "maxLength": 500
        }
    },
    "required": ["character", "backstory"],
    "additionalProperties": False
}

print("Schema validation functions created!")
print(f"Game character schema requires: {game_character_schema['required']}")

## Exercise 5: Provider Comparison Framework

Let's create a framework to compare different AI providers:

In [ ]:
def evaluate_provider_performance(client, test_prompts: List[str], schema: Dict[str, Any], provider_name: str) -> Dict[str, Any]:
    """Evaluate provider performance across multiple metrics."""
    results = {
        'provider': provider_name,
        'total_tests': len(test_prompts),
        'valid_json_count': 0,
        'schema_compliant_count': 0,
        'total_latency': 0,
        'errors': []
    }
    
    for i, prompt in enumerate(test_prompts):
        print(f"Testing prompt {i+1}/{len(test_prompts)}: {prompt[:50]}...")
        
        start_time = time.time()
        try:
            # This is a simplified version - adapt based on your client type
            if hasattr(client, 'chat_completions'):
                # OpenAI-style client
                response = client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": f"Generate a game character. {prompt}"}],
                    response_format={"type": "json_object"}
                )
                response_text = response.choices[0].message.content
            else:
                # HuggingFace-style client
                structured_prompt = f"""
                Generate a game character based on this request: {prompt}
                
                Respond with valid JSON that matches this schema:
                {json.dumps(game_character_schema, indent=2)}
                
                Output only the JSON, no additional text.
                """
                response_text = client.text_generation(structured_prompt, max_new_tokens=500)
            
            latency = time.time() - start_time
            results['total_latency'] += latency
            
            # Parse and validate response
            parsed_data = safe_json_parse(response_text)
            if parsed_data:
                results['valid_json_count'] += 1
                if validate_json_schema(parsed_data, schema):
                    results['schema_compliant_count'] += 1
            
        except Exception as e:
            results['errors'].append(str(e))
            print(f"Error: {e}")
    
    # Calculate averages and percentages
    if results['total_tests'] > 0:
        results['valid_json_rate'] = (results['valid_json_count'] / results['total_tests']) * 100
        results['schema_compliance_rate'] = (results['schema_compliant_count'] / results['total_tests']) * 100
        results['avg_latency'] = results['total_latency'] / results['total_tests']
    
    return results

# Test prompts for evaluation
test_prompts = [
    "Create a brave warrior with high health and strength.",
    "Generate an intelligent mage with powerful magic abilities.",
    "Create a stealthy rogue character.",
    "Make a wise cleric with healing powers.",
    "Generate a balanced character with moderate stats."
]

print("Provider comparison framework created!")
print(f"Test prompts prepared: {len(test_prompts)}")

## Exercise 6: Local Model Integration with Ollama

Let's test with a local Ollama instance (if available):

In [ ]:
def test_ollama_integration():
    """Test integration with local Ollama instance."""
    try:
        # Check if Ollama is running
        ollama_response = requests.get("http://localhost:11434/api/tags", timeout=5)
        if ollama_response.status_code == 200:
            print("Ollama is running!")
            models = ollama_response.json().get('models', [])
            print(f"Available models: {[model['name'] for model in models]}")
            
            # Test with a simple prompt
            test_payload = {
                "model": "llama2" if any('llama2' in m['name'] for m in models) else models[0]['name'],
                "prompt": "Generate a simple JSON object with 'name' and 'age' fields.",
                "format": "json",
                "stream": False
            }
            
            response = requests.post(
                "http://localhost:11434/api/generate",
                json=test_payload,
                timeout=30
            )
            
            if response.status_code == 200:
                result = response.json()
                print(f"Ollama response: {result.get('response', 'No response')}")
                return True
            else:
                print(f"Ollama API error: {response.status_code}")
                return False
        else:
            print("Ollama is not responding properly")
            return False
            
    except requests.exceptions.ConnectionError:
        print("Ollama is not running. Start it with: ollama serve")
        return False
    except requests.exceptions.Timeout:
        print("Ollama connection timed out")
        return False
    except Exception as e:
        print(f"Error connecting to Ollama: {e}")
        return False

# Test Ollama integration
print("Testing Ollama integration...")
ollama_available = test_ollama_integration()

## Exercise 7: Final Project - Game Character Generator

Create a complete game character generator with validation and error handling:

In [ ]:
class GameCharacterGenerator:
    """A robust game character generator with multiple provider support."""
    
    def __init__(self, provider: str = "openai", api_key: str = None):
        self.provider = provider
        self.api_key = api_key
        self.client = self._initialize_client()
    
    def _initialize_client(self):
        """Initialize the appropriate client based on provider."""
        if self.provider == "openai":
            try:
                from openai import OpenAI
                return OpenAI(api_key=self.api_key)
            except ImportError:
                raise ImportError("OpenAI library not installed")
        elif self.provider == "huggingface":
            try:
                from huggingface_hub import InferenceClient
                return InferenceClient(token=self.api_key)
            except ImportError:
                raise ImportError("HuggingFace Hub library not installed")
        elif self.provider == "ollama":
            return None  # Ollama uses HTTP requests
        else:
            raise ValueError(f"Unknown provider: {self.provider}")
    
    def generate_character(self, user_request: str) -> Dict[str, Any]:
        """Generate a game character with validation."""
        max_attempts = 3
        
        for attempt in range(max_attempts):
            try:
                if self.provider == "openai":
                    response = self._generate_openai_character(user_request)
                elif self.provider == "huggingface":
                    response = self._generate_hf_character(user_request)
                elif self.provider == "ollama":
                    response = self._generate_ollama_character(user_request)
                else:
                    raise ValueError(f"Unknown provider: {self.provider}")
                
                # Validate the response
                if validate_json_schema(response, game_character_schema):
                    return {
                        "success": True,
                        "character": response,
                        "attempts": attempt + 1
                    }
                else:
                    print(f"Attempt {attempt + 1}: Schema validation failed")
                    
            except Exception as e:
                print(f"Attempt {attempt + 1}: Error - {e}")
        
        return {
            "success": False,
            "error": "Failed to generate valid character after maximum attempts",
            "attempts": max_attempts
        }
    
    def _generate_openai_character(self, user_request: str) -> Dict[str, Any]:
        """Generate character using OpenAI."""
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{
                "role": "system",
                "content": f"Generate a game character based on the user's request. Respond with valid JSON that matches this schema: {json.dumps(game_character_schema)}"
            }, {
                "role": "user",
                "content": user_request
            }],
            response_format={"type": "json_object"}
        )
        
        response_text = response.choices[0].message.content
        return safe_json_parse(response_text)
    
    def _generate_hf_character(self, user_request: str) -> Dict[str, Any]:
        """Generate character using HuggingFace."""
        prompt = f"""
        Generate a game character based on this request: {user_request}
        
        Respond with valid JSON that matches this schema:
        {json.dumps(game_character_schema, indent=2)}
        
        Output only the JSON, no additional text.
        """
        
        response_text = self.client.text_generation(prompt, max_new_tokens=500)
        return safe_json_parse(response_text)
    
    def _generate_ollama_character(self, user_request: str) -> Dict[str, Any]:
        """Generate character using Ollama."""
        payload = {
            "model": "llama2",
            "prompt": f"Generate a game character based on this request: {user_request}",
            "format": "json",
            "stream": False
        }
        
        response = requests.post("http://localhost:11434/api/generate", json=payload)
        if response.status_code == 200:
            result = response.json()
            return safe_json_parse(result.get('response', '{}'))
        else:
            raise Exception(f"Ollama API error: {response.status_code}")

# Test the character generator
print("Testing Game Character Generator...")

# Test with different providers
providers_to_test = []
if OPENAI_API_KEY != "your-openai-key-here":
    providers_to_test.append("openai")
if HF_TOKEN != "your-huggingface-token-here":
    providers_to_test.append("huggingface")
if ollama_available:
    providers_to_test.append("ollama")

for provider in providers_to_test:
    try:
        print(f"\n--- Testing {provider.upper()} ---")
        generator = GameCharacterGenerator(provider=provider, api_key=OPENAI_API_KEY if provider == "openai" else HF_TOKEN)
        result = generator.generate_character("Create a brave warrior with high strength and good health.")
        
        if result['success']:
            print(f"✅ Character generated successfully in {result['attempts']} attempt(s)")
            character = result['character']['character']
            print(f"Name: {character['name']}")
            print(f"Class: {character['class']}")
            print(f"Health: {character['health']}")
            print(f"Strength: {character['strength']}")
        else:
            print(f"❌ Failed to generate character: {result['error']}")
            
    except Exception as e:
        print(f"Error testing {provider}: {e}")

## Summary and Next Steps

Congratulations! You've completed the Function Calling and Structured Outputs lab. Here's what you've learned:

### Key Skills Acquired:
- ✅ Function calling implementation with JSON Schema
- ✅ Cross-platform AI provider integration (OpenAI, HuggingFace, Ollama)
- ✅ Structured output validation and error handling
- ✅ Provider performance comparison and evaluation
- ✅ Building robust AI applications with retry logic

### Best Practices:
- Always validate AI outputs against schemas
- Implement proper error handling and retry mechanisms
- Use environment variables for API keys
- Test across multiple providers for reliability
- Handle JSON parsing edge cases gracefully

### Next Steps:
1. Experiment with different schemas for your use cases
2. Try integrating with other AI providers
3. Build more complex function calling scenarios
4. Implement caching for better performance
5. Add monitoring and logging for production use

### Additional Resources:
- [JSON Schema Documentation](https://json-schema.org/)
- [OpenAI Function Calling Guide](https://platform.openai.com/docs/guides/function-calling)
- [HuggingFace Inference API](https://huggingface.co/docs/api-inference/index)
- [Ollama Documentation](https://ollama.ai/)

Remember: The key to reliable AI applications is robust validation, proper error handling, and thorough testing across different scenarios and providers!